## 1. Cohort Retention & LTV

* Month‑of‑signup cohorts: % still active at 1, 3, 6 months

* Estimated Customer Lifetime Value (≈ ARPU × expected tenure) by cohort/segment

## 2. Feature & Module Use

* Top 5 features/modules by usage frequency; compare churners vs. non‑churners

* API calls, key feature events over time (seasonality, day‑of‑week)

## 3. Support Ticket Analysis

* Top 3 root‑cause keywords in tickets opened ≤ 30 days before churn

* Time‑to‑first‑response distribution; correlation with churn probability

## 4. Marketing Efficacy Deep‑Dive

* Churn rates within 90 days post‑acquisition, by channel

* Time between marketing event → first usage/subscription

## 5. Seasonality & Trends

* Month‑over‑month churn spikes/dips (e.g. year‑end, summer)

* Usage & open‑ticket volume seasonality